In [308]:
# 2021-03-12 (edited)

# Import master csv file
input_dir = './wandb_table'

# csv_name = 'wandb_export_2022-05-04T00_45_53.316+09_00.csv'
csv_name = 'descemb_transfer.csv'
output_dir = './wandb_table/pivot_table'

import pandas as pd
import numpy as np
import os

# Basic data cleaning+
data = pd.read_csv(os.path.join(input_dir,csv_name))
rundata = data.copy().round(3)

In [312]:
test_columns = [col for col in rundata.columns if 'test' in col and 'auprc' in col]

In [310]:
temp = rundata.loc[:, ['input2emb_model','pred_target', 'table', 'seed', 'src_data', 'eval_data','ratio']+test_columns]
#temp = rundata.loc[:, ['input2emb_model','pred_target', 'table', 'seed', 'src_data', 'test/eicu_auprc','test/mimic3_auprc', 'test_mimic4_auprc']]

In [314]:
test_columns

[]

In [311]:
temp

,input2emb_model,pred_target,table,seed,src_data,eval_data,ratio
0,descemb_rnn,mort,select,2024,mimic3,"[""eicu""]",10
1,descemb_rnn,los7,select,2021,mimic3,"[""eicu""]",10
2,descemb_rnn,mort,select,2023,mimic3,"[""eicu""]",10
3,descemb_rnn,readm,select,2024,mimic4,"[""mimic3""]",10
4,descemb_rnn,readm,select,2023,mimic4,"[""mimic3""]",10
5,descemb_rnn,mort,select,2022,mimic3,"[""eicu""]",10
6,descemb_rnn,los7,select,2020,mimic3,"[""eicu""]",10
7,descemb_rnn,mort,select,2021,mimic3,"[""eicu""]",10
8,descemb_rnn,los7,select,2024,eicu,"[""mimic3""]",10
9,descemb_rnn,los7,select,2023,eicu,"[""mimic3""]",10


In [283]:
temp['test_eicu_auprc'] = pd.Series()

/tmp/ipykernel_33357/2990174286.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp['test_eicu_auprc'] = pd.Series()


In [287]:
temp.src_data.value_counts()

mimic4    10
mimic3     7
eicu       7
Name: src_data, dtype: int64

In [248]:
temp['test/eicu_auprc'] = pd.Series()
temp['test/mimic3_auprc'] = pd.Series()

/tmp/ipykernel_33357/2142482442.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp['test/eicu_auprc'] = pd.Series()
/tmp/ipykernel_33357/2142482442.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp['test/mimic3_auprc'] = pd.Series()


In [288]:
temp['test/eicu_auprc'].fillna(temp['test_eicu_auprc'],inplace=True)
#temp['test/mimic3_auprc'].fillna(temp['test_mimic3_auprc'], inplace=True)

temp['test/mimic3_eicu_auprc'].fillna(temp['test_mimic3_eicu_auprc'],inplace=True)
temp['test/mimic3_mimic4_auprc'].fillna(temp['test_mimic3_mimic4_auprc'], inplace=True)
temp['test/mimic3_mimic4_eicu_auprc'].fillna(temp['test_mimic3_mimic4_eicu_auprc'],inplace=True)

temp['test/mimic4_auprc'].fillna(temp['test_mimic4_auprc'], inplace=True)
temp['test/mimic4_eicu_auprc'].fillna(temp['test_mimic4_eicu_auprc'],inplace=True)


KeyError: 'test/eicu_auprc'

In [271]:
temp['eval_data'] = temp['eval_data'].map(eval)

In [273]:
temp['eval_data'][0][0]

'eicu'

In [289]:
#temp = temp[temp['src_data']!='mimic4']
temp['auprc'] = temp.apply(lambda x: x['test/'+x['eval_data'][0]+'_auprc'], axis=1)
#temp['auprc'] = temp.apply(lambda x: x['test_'+x['src_data']+'_auprc'], axis=1)
temp = temp[['input2emb_model','pred_target','table', 'seed', 'src_data', 'auprc', 'pretrained_load']]
temp.head()

KeyError: 'test/[_auprc'

In [129]:
def max_list(x):
    
    seed_list = [np.nan]*5
    for i,j in zip(x.seed, x.auprc):
        seed_list[i-2020] = j
    return seed_list

auprc = pd.DataFrame(temp.groupby(['input2emb_model','pred_target', 'table','src_data'])[['seed', 'auprc']].apply(max_list))
#auprc.to_csv('./data/wandb/auprc.csv')


In [130]:
def max_list(x):
    
    seed_list = [np.nan]*5
    for i in x:
        seed_list[i-2020] = i
    return seed_list
    
    
seed = pd.DataFrame(temp.groupby(['input2emb_model','pred_target', 'table','src_data'])['seed'].apply(set).apply(list).apply(max_list))
#seed.to_csv('./data/wandb/seed.csv')


In [131]:
auprc.reset_index(inplace=True)
split_df = pd.DataFrame(auprc[0].tolist(), columns=['2020','2021','2022','2023','2024'])
df = pd.concat([auprc, split_df], axis=1)
df = df.drop(0, axis=1)
df.head()

,input2emb_model,pred_target,table,src_data,2020,2021,2022,2023,2024
0,codeemb,dx,whole,mimic3_eicu,0.718,0.724,0.719,0.715,0.718
1,codeemb,dx,whole,mimic3_mimic4,0.813,0.814,NaN,NaN,NaN
2,codeemb,los3,whole,mimic3_eicu,0.604,0.614,0.611,0.619,0.617
3,codeemb,los3,whole,mimic3_mimic4,0.655,0.643,0.636,0.643,0.652
4,codeemb,los3,whole,mimic4_eicu,0.607,0.602,NaN,NaN,NaN


In [75]:
# df.to_csv('./data/wandb/seed2column.csv', index=False)

In [76]:
seed.reset_index(inplace=True)
split_df = pd.DataFrame(seed['seed'].tolist(), columns=['2020','2021','2022','2023','2024'])
df = pd.concat([seed, split_df], axis=1)
df = df.drop('seed', axis=1)
df.head()

,input2emb_model,pred_target,table,src_data,2020,2021,2022,2023,2024
0,descemb,dx,select,eicu,2020,NaN,NaN,NaN,NaN
1,descemb,dx,select,mimic4,2020,2021.0,2022.0,2023.0,2024.0
2,descemb,dx,whole,mimic4,2020,2021.0,NaN,NaN,NaN
3,descemb,fi_ac,select,eicu,2020,2021.0,2022.0,NaN,NaN
4,descemb,fi_ac,select,mimic4,2020,2021.0,2022.0,2023.0,2024.0


In [27]:
df.to_csv(os.path.join(output_dir, 'DescEmb0507_df.csv'), index=False)

In [26]:
result_mean = pd.pivot_table(temp,
                       columns = [ 'table', 'input2emb_model'],
                          index = ['pred_target','src_data','seed'],
                       values = ['test_mimic3_auprc', 'test_eicu_auprc', 'test_mimic4_auprc'],
                    )
    

KeyError: 'test_mimic3_auprc'

In [54]:
temp

,input2emb_model,pred_target,table,seed,src_data,test/eicu_auprc,test/mimic3_auprc,test_eicu_auprc,test_mimic3_auprc,test_mimic3_eicu_auprc,test_mimic3_mimic4_auprc,test_mimic3_mimic4_eicu_auprc,test_mimic4_auprc,test_mimic4_eicu_auprc
0,descemb_bert,los3,whole,2024,mimic3_mimic4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,descemb,los3,whole,2022,mimic3_mimic4_eicu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,descemb_bert,los3,whole,2024,mimic3_mimic4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,descemb,los3,whole,2022,mimic3_mimic4_eicu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,descemb_bert,los3,whole,2024,mimic3_mimic4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,descemb_rnn,los3,select,2024,mimic3_mimic4,NaN,NaN,NaN,0.609,NaN,0.571,NaN,0.533,NaN
535,descemb_rnn,los3,select,2023,mimic3_mimic4,NaN,NaN,NaN,0.622,NaN,0.563,NaN,0.521,NaN
536,descemb_rnn,los3,select,2022,mimic3_mimic4,NaN,NaN,NaN,0.606,NaN,0.563,NaN,0.529,NaN
537,descemb_rnn,los3,select,2021,mimic3_mimic4,NaN,NaN,NaN,0.607,NaN,0.555,NaN,0.516,NaN


In [56]:
temp.columns

Index(['input2emb_model', 'pred_target', 'table', 'seed', 'src_data', 'auprc'], dtype='object')

In [185]:
temp.drop(columns=['test/eicu_auprc', 'test/mimic3_auprc'], inplace=True)

In [279]:
temp_df = temp[['input2emb_model', 'pred_target', 'table', 'seed', 'src_data', 'pretrained_load', 'test/eicu_auprc', 'test/mimic3_auprc', 'test/mimic4_auprc']]

KeyError: "['pretrained_load', 'test/eicu_auprc', 'test/mimic3_auprc', 'test/mimic4_auprc'] not in index"

In [300]:
import csv
f=open("train.csv","w")
wr=csv.writer(f)
wr.writerow(['input2emb_model', 'pred_target', 'table', 'seed', 'src_data', 'auprc', 'src_data', 'eval_data', 'ratio'])
f.close()

In [301]:
f=open("train.csv","a")
wr=csv.writer(f)
target_list = ['test/eicu_auprc', 'test/mimic3_auprc', 'test/mimic4_auprc']
for index,row in temp_df.iterrows():
    if row[target_list].isnull().sum() !=3:
        row.fillna(' ', inplace=True)
        rows = []
        stat = row [['input2emb_model', 'pred_target', 'table', 'seed','src_data', 'eval_data', 'ratio' ]]
        auprc =[]
        test_sample = []
        for target in target_list:
            if row[target] != ' ' :
                auprc.append(row[target])
                test_sample.append(target.split('/')[1].split('_')[0])
        rows.extend([list(stat.values) + [a] + [s]  for a, s in zip(auprc, test_sample)])
    wr.writerows(rows)
f.close()

KeyError: "['eval_data', 'ratio'] not in index"

In [235]:
df = pd.read_csv("train.csv")

In [236]:
df.input2emb_model.value_counts()

descemb_bert    33
descemb          6
Name: input2emb_model, dtype: int64

In [223]:
tmp = df[df['input2emb_model'] =='codeemb_bert']

In [237]:
result_mean = pd.pivot_table(df,
                       columns = ['test_sample','table', 'input2emb_model', 'src_data'],
                          index = ['pred_target', 'seed'],
                       values = ['auprc'],
                       aggfunc=['mean']
                    )
    

In [238]:
result_mean

mean                                        \
                              auprc                                         
test_sample                    eicu                                mimic3   
table                        select              whole             select   
input2emb_model             descemb       descemb_bert            descemb   
src_data         mimic3_mimic4_eicu mimic3_mimic4_eicu mimic3_mimic4_eicu   
pred_target seed                                                            
dx          2020                NaN             0.3885                NaN   
            2021                NaN             0.4035                NaN   
            2023              0.662             0.4020              0.762   
            2024              0.654                NaN              0.758   

                                                                           
                                                                           
test_sample                                     mimic4                     
table                         whole             select              whole  
input2emb_model        descemb_bert            descemb       descemb_bert  
src_data         mimic3_mimic4_eicu mimic3_mimic4_eicu mimic3_mimic4_eicu  
pred_target seed                                                           
dx          2020              0.621                NaN           0.688000  
            2021              0.613                NaN           0.697167  
            2023              0.617              0.831           0.689000  
            2024                NaN              0.832                NaN

In [226]:
result_mean.to_csv('/home/ghhur/wandb_table/result_table/codeemb_pooled_codeemb_bert_0511.csv')

In [13]:
# def generate_table(result_file:pd.DataFrame, seed_count=False, count_flag=True, std=False, rounding=True):
    
    
#     # Possible combinations for our models
#     conditionlist = [(result_file['bert_induced'] == False),
#                      (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
#                              (result_file['cls_freeze'] == True),
#                      (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == True) & 
#                              (result_file['cls_freeze'] == False),
#                      (result_file['bert_induced'] == True) & (result_file['bert_freeze'] == False)]
#     choicelist = ['singleRNN', 'DescEmb_FR', 'DescEmb_FT','BERTfinetune']
#     result_file['model_name'] = np.select(conditionlist, choicelist)
    
    
    
#     result_file = result_file.loc[:, ['model_name','bert_model','bert_induced','seed', 
#                                       'source_file', 'target','item','mimic_test_auprc', 'eicu_test_auprc']]
    
#     result_filter = result_file.copy() #.loc[result_file['target']==target]
     
#     result_mean = pd.pivot_table(result_filter,
#                                columns = ['source_file', 'model_name'],
#                                index = ['item','target','bert_induced', 'bert_model'],
#                                values = ['mimic_test_auprc', 'eicu_test_auprc'],
#                                aggfunc=['mean'])
    
#     result_std = pd.pivot_table(result_filter,
#                                columns = ['source_file', 'model_name'],
#                                index = ['item','target','bert_induced', 'bert_model'],
#                                values = ['mimic_test_auprc', 'eicu_test_auprc'],
#                                aggfunc=['std'])
    
#     result_count = pd.pivot_table(result_filter,
#                                columns = ['source_file', 'model_name'],
#                                index = ['item','target','bert_induced', 'bert_model'],
#                                values = ['mimic_test_auprc', 'eicu_test_auprc'],
#                                aggfunc=['count'])
    
#     if count_flag == True:
#         if rounding==True:
#             RM_drop = result_mean.droplevel([0], axis=1).round(3)
#             RS_drop = result_std.droplevel([0], axis=1).round(3)
#         elif rounding == False:
#             RM_drop = result_mean.droplevel([0], axis=1)
#             RS_drop = result_std.droplevel([0], axis=1)
#         RC_drop = result_count.droplevel([0], axis=1)
        
#         RC_drop = RC_drop[RM_drop.columns]
        
#         for col in range(len(RM_drop.columns)): 
#             RM_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
#             RM_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'   
            
#             RS_drop.iloc[:,col][RC_drop.iloc[:,col] > 10] = 'over'
#             RS_drop.iloc[:,col][RC_drop.iloc[:,col] < 10] = 'under'  
    
    
#     if seed_count==False:
#         if count_flag == True:
#             if std==True:
#                 return RS_drop
#             elif std==False:
#                 return RM_drop
#         elif count_flag == False:
#             if std==True:
#                 return result_std
#             elif std==False:
#                 return result_mean
    
#     elif seed_count==True:
#         return RC_drop



In [14]:
# generate_table(rundata, count_flag=True)